In [118]:
print("Hello from the GCP World")

Hello from the GCP World


This is a small 2 dimension model trained on a single csv cheet from the Webspoofing attack used for testing my Django app. I have left it in the submission folder as I will use it in the testing video due to the short run time but the results are not important to the overall study

In [119]:
readFile="gs://birkbeckwifibucket/parquet/smallWebSiteSpoofingTestFile"

In [120]:
! PYTHONHASHSEED=0
import pandas as pd
import numpy as np

In [121]:
dataset=pd.read_parquet(readFile, engine='pyarrow')

In [122]:
 from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [123]:
categoricalCols = [
    "wlan_da",
    "wlan_sa"
]


In [124]:
numericCols=[

]

In [125]:
y=dataset['label']
x=dataset[categoricalCols+numericCols]
x[categoricalCols].astype("category")

,wlan_da,wlan_sa
0,NA,NA
1,88:66:a5:55:a2:d4,74:d0:2b:7c:5a:5e
2,NA,NA
3,NA,NA
4,NA,NA
...,...,...
49995,50:3e:aa:e4:01:93,04:ed:33:e0:24:82
49996,NA,NA
49997,NA,NA
49998,NA,NA


In [126]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
from sklearn.preprocessing import FunctionTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categoricalCols)
    ]
)

In [127]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


In [128]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [129]:
RF = Pipeline(steps=[("preprocess", preprocessor), ("RFclassifier", RandomForestClassifier(max_depth=None, min_samples_split=2,max_features='log2',criterion='gini',n_estimators=100,random_state=0))])

In [130]:
%%time
RF.fit(X_train, y_train)
print("The model score for a RandomForestClassifier: %.3f" % RF.score(X_test, y_test))
testPredictions=RF.predict(X_test)

The model score for a RandomForestClassifier: 1.000
CPU times: user 1.05 s, sys: 7.36 ms, total: 1.05 s
Wall time: 1.81 s


In [131]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, testPredictions).ravel()
precision,recall,fscore,support = precision_recall_fscore_support(y_test, testPredictions, average='binary')
MCC = matthews_corrcoef(y_test, testPredictions)
print("There are "+str(tn)+ " true negatives")
print("There are "+str(tp)+ " true positives")
print("There are "+str(fn)+ " false negatives")
print("There are "+str(fp)+ " false positives")
print("The precision is "+str(precision))
print("The recall is "+str(recall))
print("The fscore is "+str(fscore))
print("The Matthews correlation coefficient is "+str(MCC))

There are 7031 true negatives
There are 2969 true positives
There are 0 false negatives
There are 0 false positives
The precision is 1.0
The recall is 1.0
The fscore is 1.0
The Matthews correlation coefficient is 1.0


In [132]:
from google.cloud import storage
import joblib
storage_client = storage.Client()
model_filename = "smallmodel.joblib"
with open(model_filename, "wb") as model_file:
    joblib.dump(RF, model_file)
    
bucket_name="birkbeckwifibucket"
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(model_filename)
blob.upload_from_filename(model_filename)